In [1]:
import geopandas as gp
%matplotlib inline
import shapely.geometry as shpgeo
PLACE_POLYS_MUSEUM = '../data/place_polys_museum.geojson' 
PLACE_POLYS_NP = '../data/place_polys_np.geojson'
# polys = gp.read_file(PLACE_POLYS_MUSEUM)
polys = gp.read_file(PLACE_POLYS_NP)


In [2]:
min_lat= 59.066574
max_lon = -8.089235 
max_lat = 50.126911
min_lon = 1.793197
lon_0 = (max_lon+min_lon)/2
lat_0 = (max_lat+min_lat)/2
eng_bbox = shpgeo.Polygon([(max_lon,min_lat), (min_lon,min_lat), (min_lon,max_lat), (max_lon, max_lat)])
polys.geometry.apply(lambda x: x.within(eng_bbox)).value_counts()

False    83
dtype: int64

In [3]:
# uk museum
target_crs = {'datum':'WGS84', 'no_defs':True, 'proj':'aea', 'lat_1':min_lat, 'lat_2':max_lat, 'lat_0':lat_0, 'lon_0':lon_0} 
# usa national park
target_crs = {'proj': 'aea' ,'lat_1':29.5,'lat_2':45.5,'lat_0':37.5 ,'lon_0':-96 ,
           'x_0':0, 'y_0':0, 'datum':'NAD83', 'units':'m', 'no_defs':True}
polys['area'] = polys.to_crs(crs=target_crs).geometry.apply(lambda x: x.area)

In [4]:
def clean_place(x):
    if '##' in x:
        x = x.split('##')[0]
    x = x[:-1] if x[-1].isdigit() else x
    return x.replace(' ','_')

In [5]:
polys.place = polys.place.apply(clean_place)

In [6]:
polys['area'].order()

32    1.248603e+02
31    1.420875e+02
51    7.117263e+02
33    8.407334e+02
73    1.191251e+03
40    1.373043e+03
8     3.098034e+03
35    5.027369e+03
72    5.416467e+03
21    6.820164e+03
14    8.333892e+03
13    1.289262e+04
27    1.413327e+04
39    1.764731e+04
20    1.908648e+04
37    2.045945e+04
5     2.051696e+04
12    2.170807e+04
29    3.204933e+04
17    5.261821e+04
34    5.894206e+04
7     6.797724e+04
24    8.995106e+04
19    1.062955e+05
10    1.086532e+05
26    1.290770e+05
4     1.836641e+05
36    2.061719e+05
6     2.096805e+05
25    2.370531e+05
          ...     
47    7.719014e+06
41    1.173285e+07
0     1.240238e+07
30    1.372380e+07
71    3.424443e+07
15    3.970652e+07
70    4.189874e+07
18    8.619568e+07
55    1.030685e+08
67    1.249508e+08
56    1.331683e+08
74    1.456341e+08
82    1.706828e+08
52    2.034580e+08
69    2.386897e+08
64    2.597006e+08
81    2.967572e+08
49    3.087400e+08
78    3.737744e+08
76    3.764839e+08
63    4.957274e+08
80    5.0770

In [10]:
places = polys.groupby('place').agg(sum).reset_index()[['place','area']]


In [11]:
places

,place,area
0,Acadia_NP_(ACAD),1.590927e+08
1,Adams_NHP,1.764731e+04
2,African_Burial_Ground_NM_(AFBG),1.373043e+03
3,Agate_Fossil_Beds_NM,1.173285e+07
4,Alibates_Flint_Quarries_NM_(ALFL),5.578203e+06
5,Allegheny_Portage_Railroad_NHS_(ALPO),3.049101e+05
6,Andersonville_NHS_(ANDE),2.044381e+06
7,Aniakchak_NM_&_PRES_(ANIA),1.409972e+09
8,Antietam_NB_(ANTI),7.719014e+06
9,Appomattox_Court_House_NHP_(APCO),5.899875e+06


In [12]:
path = '../data/place_np_area.csv'
places.to_csv(path)

In [94]:
def haversine(lon1, lat1, lon2, lat2):
    from math import radians, cos, sin, asin, sqrt
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    m = km *1000
    return m   

def grid_line(mini, maxi, ngrid=10):
    delta = (maxi-mini)/ngrid
    return [(mini+i*delta, mini+(i+1)*delta) for i in range(ngrid)] 


def grid_bbox(sw, ne, ngrid=10):
    grid_lat = grid_line(sw[0], ne[0], ngrid)
    grid_lon = grid_line(sw[1], ne[1], ngrid)
    grids = []
    for i in range(ngrid):
        for j in range(ngrid):
            s, n = grid_lat[i]
            w, e = grid_lon[j]
            grids.append(((s,w),(n,e)))
    return grids
def area_of_grid(grid):
    (s,w), (n,e) = grid
    l = haversine(w,s,e,s)
    w = haversine(e,n,e,s)
    return l*w

def swne2poly(sw,ne):
    return shpgeo.box(sw[1], sw[0], ne[1], ne[0])

def area_poly(polygon, ngrid=100):
    w,s,e,n = polygon.bounds
    polygon.within(shpgeo.Polygon([(w,s),(e,s),(e,n),(w,n)]))
    grids = grid_bbox((s,w),(n,e),ngrid)
    unit_area = area_of_grid(grids[0])
    grids = [swne2poly(*grid)for grid in grids]
    grids = [grid for grid in grids if grid.intersects(polygon)]
    return len(grids)*unit_area